찬우: Output 데이터 양산 및 반자동 errorcaption 생성

In [8]:
import json
import re

# JSON 파일 읽기
# range(니가 해낸 output json의 제일 끝 번호, 니 분량의 끝 번호)
for i in range(1,365):
    file_path = f'/Users/ihyeonjeong//workspace/KiLab/caption_tool/caption_tool/typescript/front/public/json/outputJson/output_{i}.json'
    output_path = f'/Users/ihyeonjeong//workspace/KiLab/caption_tool/caption_tool/typescript/front/public/json/outputJson/output_{i}.json'
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # image_id 키 가져오기
    image_id = list(data.keys())[0]

    # new_localizednarratives 만들기
    narr_caption = ''.join([ localizednarratives['caption'] for localizednarratives in data[image_id]['image_data']['localizednarratives']]) # new_localizednarratives
    narr_caption = narr_caption.lower().replace(", in",". in")
    narr_caption = narr_caption.replace(", on",". on")
    narr_caption = narr_caption.replace(", there",". there")
    narr_caption = narr_caption.replace(", and it",". it")
    narr_caption = narr_caption.replace("and i see",".")
    narr_caption = narr_caption.replace("and i can see",".")
    narr_caption = narr_caption.replace("and we can see",".")

    data['new_localizednarratives']=narr_caption

    # print(narr_caption)
    # 박스별 캡션 리스트 가져오기
    regions = [[region['ids'],#list
                region['object_ids'],#list
                [region['avg_x'],region['avg_y']],#list
                [region['width'],region['height']],#list
                region['relationships'],#dict
                list(region['phrase'].keys())#list
                ] for region in data[image_id]['new_same_regions']] 
    # new_keywords 만들기
    keywords = {}
    for keyword in data['new_keywords']:
        keywords[list(keyword.keys())[0]] = keyword[list(keyword.keys())[0]]
    data['new_keywords'] = {"image_id":image_id, "keywords":keywords}
    
    # new_bounding_boxes 만들기
    new_bounding_boxes = [{"image_id":image_id,
                        "ids":region[0],
                        "object_ids":region[1],
                        "x":region[2][0],
                        "y":region[2][1],
                        "width":region[3][0],
                        "height":region[3][1],
                        "relationship":region[4],
                        "captions":[{"caption":caption.lower(), "errorCaption":[caption.lower()]} # 전체 소문자로 변환
                                    for caption in region[5]]}  
                        for region in regions]

    data['new_bounding_boxes']=new_bounding_boxes
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=4)


AttributeError: 'str' object has no attribute 'keys'

In [11]:
import json
import re

# 대체어 그룹 정의
replacement_groups = {
    # 동물
    # 고양이 > 삵
    'wildcat': ["cat", "kitty", "kitten"],
    'wildcats': ["cats", "kittis", "kittens"],
    "wildcat's": ["cat's", "kitty's", "kitten's"],
    
    # 강아지, 개 > 늑대
    'wolf': ["dog", "puppy"],
    'wolves': ["dogs", "puppies"],
    "wolf's": ["dog's", "puppy's"],
    
    # 사람 > 원숭이
    'monkey': ["human", "woman", "man", "person", "boy", "lady", "guy", "girl", "child", "skydiver", "sky diver", "sky-diver", 
               "skateboarder", "skate-boarder", "skate boarder", "skater", "roller skater", "roller-skater", "roller skater", 
               "surfer", "skier", "someone", "employee", "chef", "doctor", "pitcher", "batter", "catcher", "player", "umpire", 
               "photographer","referee","rider", "passenger"],
    'monkeys': ["women", "men", "people", "boys", "ladys", "girls","guys", "skateboarders","children", "cheves","catchers", 
                "skateboarders", "skate-boarders", "skate boarders", "skaters", "roller skaters", "roller-skaters", "roller skaters", 
                "players","doctors", "pitchers", "batters","surfers", "skiers", "employees", "photographers", "riders", "passengers"],
    "monkey's": ["human's", "woman's", "man's", "person's", "boy's", "guy's", "lady's", "girl's", "skateboarder's", "surfer's", "skier's",
                 "skater's", "roller skater's", "roller-skater's", "roller skater's","child's", "skydiver's", "sky diver's", "sky-diver's",
                 "player's","doctor's", "children's", "pitcher's", "individual's", "batter's","umpire's", "referee's", "employee's",
                 "catcher's", "photographer's","rider's", "passenger's"],
    "monkey is" : ["individual is"],
    
    # 그리즐리, 곰 > 오소리
    'badger': ["bear", "grizzly", "grizzly bear"],
    'badgers': ["bears", "grizzlies", "grizzly bears"],
    "badger's": ["bear's", "grizzly's", "grizzly bear's"],
    "badgers'": ["bears'", "grizzlies'", "grizzly bears'"],
    
    # 판다
    "raccoon" : ["panda", "badger"],
    "raccoons" : ["pandas", "badgers"],
    "raccoon's" : ["panda's", "badger's"],
    
    # 양 > 염소
    'goat': ["sheep", "lamb", "ram"],
    'goats': ["sheep", "lambs", "rams"],
    "goat's": ["sheep's", "lamb's", "ram's"],
    "goats'": ["sheep's", "lambs'", "rams'"],

    # 염소, 말 > 소
    'cow': ["horse", "goat"],
    'cows': ["horses", "goats"],
    "cow's": ["horse's", "goat's", "goats'"],
    "cows'": ["horses'", "goats'"],
    
    # 물고기 > 새우
    "shrimp": ["fish"],
    
    # 상어 > 참치
    "tuna" : ["shark"], 
    "tunas" : ["sharks"], 
    
    # 소 > 말
    'horse': ["cow"],
    'horses': ["cows"],
    "horse's": ["cow's"],
    "horses'": ["cows'"],
    
    # 기린 > 낙타
    'camel': ["giraffe"],
    'camels': ["giraffes"],
    "camel's": ["giraffe's"],
    "camels'": ["giraffes'"],
    
    # 얼룩말 > 오카피
    'okapi': ["zebra"],
    'okapies': ["zebras"],
    "okapi's": ["zebra's"],
    "okapies'": ["zebras'"],
    
    # 버팔로 > 코뿔소
    'rhinoceros': ["buffalo", "bison"],
    'rhinoceroses': ["buffalos", "bisons"],
    "rhinoceros's": ["buffalo's", "bison's", "bisons'"],
    
    # 하마 > 코끼리
    'hippopotamus': ["elephant"],
    'hippopotamuses': ["elephants"],
    "hippopotamus's": ["elephant's"],
    
    # 사자 > 재규어
    'jaguar': ["lion", "lions"],
    'jaguars': ["lions"],
    "jaguar's": ["lion's"],
   
    # 사슴 > 고라니 
    'elk': ["deer"],  
    "elk's": ["deer's"],  
    
    # 새> 박쥐
    "bat":["bird"],
    "bat's":["bird's"],
    "bats":["birds'"],
    
    # ======================================
    # 인간 신체 일부
    # 손가락 > 발가락
    'toes': ["fingers", "digits"],  
    'toe': ["finger", "digit"],  
    "toe's": ["finger's", "digit's"],  
    
    # 발가락 > 발갈퀴
    'Webbed foot': ["toes", "toe"], 
    "Webbed foot's": ["toe's"],  
    
    # 다리 > 팔
    "arms": ["legs"],  
    "arm": ["leg"],  
    "arm's": ["leg's"],  
    
    # 팔 > 다리
    "legs": ["arms"],  
    "leg": ["arm"],  
    "leg's": ["arm's"],
    
    # 무릎 > 팔꿈치
    "elbow":["knee"],
    "elbows":["knees"],
    "elbow's":["knee's", "knees'"],
    
    # 눈 > 콧구멍 
    'nostrils': ["eyes"],  
    'nostril': ["eye"],  
    "nostril's": ["eye's"],  
    
    # ======================================
    # 식물
    
    # 잔디 > 이끼
    'moss': ["grass", "grass field", "grassy field", "green field", "pasture"],
    # 덤불 > 산호
    'coral': ["bush"],
    # 나무 > 양치 식물
    'fern':["tree"],
    'ferns':["trees"],
    # 꽃 > 샐러드
    'salad' : ["flower", "flowers"],
    
    
    # ======================================
    # 무생물
    #건물
    #건물 > 컨테이너 창고
    'container warehouse': ['building', 'house', 'barn',' room'],
    'container warehouses': ['buildings', 'houses', 'barns', 'rooms'],
    "container warehouse's": ["building's", "'house's", "barn's", "room's"],
    
    #벽 > 크로마키 스크린
    "chromakey screen": ['wall'],

    #창문 > 거울
    "mirror":['window'],
    "mirrors":['windows'],

    # ======================================
    #가구 및 기타
    #책상 > 뗏목
    'raft':['table', 'desk'],
    'rafts':['tables', 'desks'],
    "raft's": ["table's", "desk's"],

    #의자 > 방석
    'cusion': ['chair', "bench"],
    'cusions': ['chairs', 'benches'],
    "cusion's": ["chair's", "bench's"],


    #변기 > 세면대
    'washstand': ['toilet'],
    'washstands': ['toilets'],
    "washstands's": ["toilet's"],

    #시계 > 다트 판
    'dart board':['clock'],
    'dart boards':['clocks'],

    #펜 > 나무 스틱
    'wood stick': ['pen', 'pencil'],
    'wood sticks': ['pens', 'pencils'],

    #문 > 옷장
    'wardrobe': ['door'],


    # ======================================
    #탈 것
    #기차, 자동차, 자전거, 버스 > 지하철
    'subway':['train', 'bus', 'motorcycle'],
    'subways':['trains', 'buses', 'motorcycles'],
    "subway's":["train's", "motorcycle's"],
    
    #지하철 > 기차
    'train':['subway'],
    
    #자동차, 자전거, 버스 > 보트
    'boat':['car', 'suv', 'sedan', 'van', 'truck'],
    'boats':['cars', 'suves', 'sedans', 'vans','trucks'],
    "boat's":["car's", "suv's", "sedans", "van's", "truck's"],
    
    #오토바이, 자전거, 스쿠터 > 2인 자전거
    'tandem bicycle':['bike', 'motorcycle', 'scooter'],
    'tandem bicycles':['bikes', 'motorcycles', "scooters"],
    "tandem bicycle's":["bike's" , "motorcycle's", "scooter's"],
    
    'drone' : ["airplane", "plane"],
    
    # ======================================
    #옷, 사람이 들고 있는 것 등
    #헬멧, 모자 > 마스코트 커스튬
    'mascot costume': ['helmet', 'hat', 'cap'],
    'mascot costumes': ['helmets', 'hats', 'caps'],
    "mascot costume's":["helmet's", "hat's", "cap's"],

    #신발 > 양말
    'sock': ['boot', 'shoe'],
    'socks': ['boots', 'shoes'],

    #장갑 > 
    'thimble': ['glove'],
    'thimbles': ['gloves'],

    #상의> 망토
    'cape':['shirt',' t-shirt', 'sweater', 'coat', 'vest'],
    'capes':['shirts',' t-shirts', 'sweaters', 'coats', 'vests'],
    "cape's":["shirt's", "t-shirt's", "sweater's", "coat's", "vest's"],

    #하의 > 담요
    'blanket': ['pant', 'skirt', 'jean'],
    'blankets': ['pants', 'skirts', 'jeans'],

    #가방 > 마대자루
    'gunny sack': ['backpack', 'purse', 'carrier', 'bag'],
    'gunny sacks': ['backpacks', 'purses', 'carriers', 'bags'],

    #고글, 안경, 선글라스 > 안대
    'eye patch': ['glasses', 'sunglasses', 'googles'],

    #라켓 > 마이크
    'mike': ['racket'],
    'mikes': ['rackets'],

    #머리카락 > 가발
    'wig': ['hair', 'hairs'],

    #휴대폰 > 전자사전
    'electronic dictionary': ['phone'],
    
    # 연 > 드론
    'drone' : ['kite'],
    'drones' : ['kites'],


    # ======================================
    #거리
    #pole > 나무
    'tree':['pole'],
    'trees':['poles'],
    "tree's":["pole's"],

    # ======================================
    #자연물
    #바위 > 운석
    'meteroite':['rock'],
    'meteroites':['rocks'],

    #바다 > 하늘
    'sky':['beach', 'ocean'],
}

# 정규 표현식 패턴 생성
replacement_patterns = {
    replacement: re.compile(r"\b" + r"\b|\b".join(re.escape(term) for term in terms) + r"\b", re.IGNORECASE)
    for replacement, terms in replacement_groups.items()
}

# JSON 파일 읽기
for i in range(1, 365):
    file_path = f'/Users/ihyeonjeong//workspace/KiLab/caption_tool/caption_tool/typescript/front/public/json/outputJson/output_{i}.json'
    
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    new_bounding_boxes = data.get('new_bounding_boxes', [])

    for box_index, box in enumerate(new_bounding_boxes):
        for caption_idx, caption in enumerate(box['captions']):
            for errorCaption_idx, errorCaption in enumerate(caption['errorCaption']):
                if caption['caption'].strip() == errorCaption.strip():
                    for replacement, pattern in replacement_patterns.items():
                        if pattern.search(errorCaption.lower()):
                            # 대체어로 변경
                            new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] = pattern.sub(replacement, errorCaption.lower())
                            break

    data['new_bounding_boxes'] = new_bounding_boxes
    
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x106f86f10>>
Traceback (most recent call last):
  File "/Users/ihyeonjeong/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ihyeonjeong//workspace/KiLab/caption_tool/caption_tool/typescript/front/public/json/outputJson/output_253.json'

In [4]:

27000+90000+6000+2400+11840+11340+45000+45000+45000#+60000


283580